# L8d: Let's Implement a Cross Validation Algorithm for Ridge Regression
In this lab, we'll explore the how to estimate the optimal regularization parameter for Ridge Regression using cross-validation. Ridge Regression is a type of linear regression that includes a regularization term to prevent overfitting by penalizing large coefficients.

> __Learning Objectives:__
> 
> By the end of this lab, you will be able to:
> Three learning objectives here

Let's get started!
___